Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [11]:
from random import choice

In [12]:
def print_table():
    global TABLE
    t2 = [p for p in TABLE if p]
    max_height = max(len(p) for p in t2)
    dim_block = max(max(len(e) for e in l) for l in t2)
    printout = list()
    for p in TABLE:
        tmp = p + [" " * dim_block] * (max_height - len(p))
        printout.append(tmp[::-1])
    for line in zip(*printout):
        print("    ".join(f"{{:{dim_block}s}}".format(b) for b in line))
    print("    ".join(f"{{:{dim_block}d}}".format(i) for i in range(len(TABLE))))

In [ ]:
TAG = ">"
TABU = list()

TABLE = None
HAND = None


def find_pile_of(block):
    return next(i for i, p in enumerate(TABLE) if block in p)


def find_pile_tabu():
    acceptable_piles = [i for i, p in enumerate(TABLE) if all(t not in p for t in TABU)]
    if not acceptable_piles:
        return -1
    else:
        return choice(acceptable_piles)


def on_top(block):
    return TABLE[find_pile_of(block)][-1] == block


def put_on(block1, block2, depth=1):
    global TABU
    TABU = [block1, block2]
    print(TAG * depth + " " + f"Putting '{block1}' on '{block2}'")
    if not on_top(block1):
        clear_above(block1, depth=depth + 1)
    if not on_top(block2):
        clear_above(block2, depth=depth + 1)
    grasp(block1, depth=depth + 1)
    drop_on_pile(find_pile_of(block2), depth=depth + 1)


def drop_on_pile(pile, depth=1):
    global HAND
    assert HAND is not None
    if pile < 0:
        pile = len(TABLE)
        TABLE.append([])
    print(TAG * depth + " " + f"Dropping block '{HAND}' on pile {pile}")
    TABLE[pile].append(HAND)
    HAND = None


def grasp(block, depth=1):
    global HAND
    assert HAND is None
    print(TAG * depth + " " + f"Grasping block '{block}'")
    pile = find_pile_of(block)
    if not on_top(block):
        clear_above(block, depth=depth + 1)
    assert TABLE[pile].pop() == block
    HAND = block


def clear_above(block, depth=1):
    print(TAG * depth + " " + f"Clearing above block '{block}'")
    pile = find_pile_of(block)
    while TABLE[pile][-1] != block:
        tmp = TABLE[pile][-1]
        grasp(tmp, depth=depth + 1)
        drop_on_pile(find_pile_tabu(), depth=depth + 1)

In [14]:
TABLE = list()
TABLE.append(["A1", "A2"])
TABLE.append(["B1", "B2", "B3"])
TABLE.append(["C1", "C2", "C3", "C4"])
TABLE.append(["D1", "D2", "D3"])
print_table()

            C4      
      B3    C3    D3
A2    B2    C2    D2
A1    B1    C1    D1
 0     1     2     3


In [15]:
put_on('C1', 'A2')
print_table()

> Putting 'C1' on 'A2'
>> Clearing above block 'C1'
>>> Grasping block 'C4'
>>> Dropping block 'C4' on pile 3
>>> Grasping block 'C3'
>>> Dropping block 'C3' on pile 3
>>> Grasping block 'C2'
>>> Dropping block 'C2' on pile 1
>> Grasping block 'C1'
>> Dropping block 'C1' on pile 0
                  C3
      C2          C4
C1    B3          D3
A2    B2          D2
A1    B1          D1
 0     1     2     3


In [16]:
put_on('D1', 'B1')
print_table()

> Putting 'D1' on 'B1'
>> Clearing above block 'D1'
>>> Grasping block 'C3'
>>> Dropping block 'C3' on pile 2
>>> Grasping block 'C4'
>>> Dropping block 'C4' on pile 2
>>> Grasping block 'D3'
>>> Dropping block 'D3' on pile 0
>>> Grasping block 'D2'
>>> Dropping block 'D2' on pile 0
>> Clearing above block 'B1'
>>> Grasping block 'C2'
>>> Dropping block 'C2' on pile 0
>>> Grasping block 'B3'
>>> Dropping block 'B3' on pile 0
>>> Grasping block 'B2'
>>> Dropping block 'B2' on pile 2
>> Grasping block 'D1'
>> Dropping block 'D1' on pile 1
B3                  
C2                  
D2                  
D3                  
C1          B2      
A2    D1    C4      
A1    B1    C3      
 0     1     2     3
